In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [2]:
org = pd.read_csv('data/Organizations.csv')
categories = pd.read_csv('data/ProductCategories.csv')
items = pd.read_csv('data/ProductItems.csv')
products = pd.read_csv('data/Products.csv')
receipts = pd.read_csv('data/Receipts.csv')
users = pd.read_csv('data/Users.csv')

In [3]:
print(f"Receipts: {receipts.columns}")
print(f"Organizations: {org.columns}")
print(f"Product Categories: {categories.columns}")
print(f"Product Items: {items.columns}")
print(f"Products: {products.columns}")

Receipts: Index(['id', 'receipt_id', 'ico', 'cash_register_code', 'issue_date',
       'create_date', 'customer_id', 'dic', 'ic_dph', 'invoice_number', 'okp',
       'paragon', 'paragon_number', 'receipt_number', 'type', 'tax_base_basic',
       'tax_base_reduced', 'total_price', 'free_tax_amount',
       'vat_amount_basic', 'vat_amount_reduced', 'vat_rate_basic',
       'vat_rate_reduced', 'exemption', 'org_building_number', 'org_country',
       'org_dic', 'org_ic_dph', 'org_ico', 'org_municipality', 'org_name',
       'org_postal_code', 'org_property_registration_number',
       'org_street_name', 'org_vat_payer', 'unit_building_number',
       'unit_cash_register_code', 'unit_country', 'unit_municipality',
       'unit_postal_code', 'unit_property_registration_number',
       'unit_street_name', 'unit_name', 'unit_type', 'pkp', 'category',
       'organization_id', 'org_unit_id', 'created_date', 'last_modified_date',
       'created_by', 'last_modified_by'],
      dtype='object')
O

In [4]:
receipts = receipts.dropna(axis=1, thresh=(len(receipts) * 0.5))

In [5]:
cor_receipts = receipts[['id', 'organization_id', 'issue_date', 'total_price', 'category', 
                         'org_name', 'org_country', 'org_municipality', 'org_street_name',]]

In [6]:
org = org.dropna(axis=1, thresh=(len(org) * 0.5))

In [7]:
cor_org = org[['id', 'name', 'country', 'municipality', 'street_name', 'category']]

In [8]:
receipts_merged = cor_receipts.merge(cor_org, left_on='organization_id', right_on='id', suffixes=('_receipt', '_org'))

In [9]:
receipts_merged['issue_date'] = pd.to_datetime(receipts_merged['issue_date'], format='%d.%m.%Y %H:%M:%S')

In [10]:
mask = receipts_merged['category_receipt'].isna()
receipts_merged.loc[mask, 'category_receipt'] = receipts_merged.loc[mask, 'category_org']
receipts_merged['category_receipt'] = receipts_merged['category_receipt'].fillna('Unknown')

In [11]:
receipts_merged = receipts_merged.drop(columns=['org_name', 'org_country', 'org_municipality', 
                                                'org_street_name', 'id_org', 'category_org'])

In [12]:
receipts_merged = receipts_merged.rename(columns={'issue_date': 'date', 
                                                  'total_price': 'price', 'category_receipt': 'category'})

In [13]:
receipts_merged.head(5)

,id_receipt,organization_id,date,price,category,name,country,municipality,street_name
0,1,1,2022-04-02 15:09:57,39.56,Stravovanie/null,BILLA s.r.o.,Slovensko,Bratislava - mestská časť Ružinov,Bajkalská
1,2,2,2022-03-05 13:56:16,71.82,Stravovanie/Potraviny,Kaufland Slovenská republika v.o.s.,Slovensko,Bratislava - mestská časť Nové Mesto,Trnavská cesta
2,3,1,2022-03-05 14:36:15,43.35,Stravovanie/Potraviny,BILLA s.r.o.,Slovensko,Bratislava - mestská časť Ružinov,Bajkalská
3,4,1,2022-02-26 14:44:56,35.27,Stravovanie/null,BILLA s.r.o.,Slovensko,Bratislava - mestská časť Ružinov,Bajkalská
4,5,3,2023-09-29 11:32:10,168.36,Stravovanie/null,"Lidl Slovenská republika, s.r.o.",Slovensko,Bratislava - mestská časť Ružinov,Ružinovská


In [14]:
print(products.columns)
print(categories.columns)
print(items.columns)

Index(['id', 'name', 'item_type', 'price', 'vat_rate', 'category',
       'organization_id', 'org_unit_id', 'created_date', 'last_modified_date',
       'created_by', 'last_modified_by', 'is_overridden'],
      dtype='object')
Index(['id', 'category', 'product_id'], dtype='object')
Index(['id', 'quantity', 'product_id', 'fs_receipt_id', 'created_date',
       'last_modified_date', 'created_by', 'last_modified_by', 'discount_id'],
      dtype='object')


In [15]:
cor_products = products[['id', 'name', 'item_type', 'price', 'category', 'organization_id']]

In [16]:
cor_items = items[['id', 'quantity', 'product_id', 'fs_receipt_id']]

In [17]:
merged_products = cor_items.merge(cor_products, left_on='product_id', right_on='id', 
                                  how='left', suffixes=('_item', '_product'))

In [18]:
merged_products = merged_products.drop_duplicates(
    subset=['product_id', 'quantity', 'fs_receipt_id', 'name'],
    keep='first'
)


In [19]:
merged_products = merged_products.drop(columns=['id_product'])
merged_products = merged_products.rename(columns={'fs_receipt_id': 'receipt_id'})

In [20]:
sales = merged_products.merge(receipts_merged, left_on='receipt_id', right_on='id_receipt', how='left', suffixes=('_product', '_receipt'))

In [21]:
def clean_org_name(name: str) -> str:
    if pd.isna(name):
        return name
    name = re.sub(
        r'\b(s\s*\.?\s*r\s*\.?\s*o\.?|a\s*\.?\s*s\.?|v\s*\.?\s*o\s*\.?\s*s\.?|spol\s*\.?\s*s\s*r\s*\.?\s*o\.?)\b',
        '',
        name,
        flags=re.IGNORECASE
    )
    name = re.sub(r'\b(Slovenská republika|Slovensko|SR|Slovakia|SK)\b', '', name, flags=re.IGNORECASE)
    name = re.sub(r'[.,/]', ' ', name)
    name = re.sub(r'\s{2,}', ' ', name)
    name = name.strip(" -_")
    return name

In [22]:
sales['municipality'] = sales['municipality'].str.split('-').str[0].str.strip()
sales['name_receipt'] = sales['name_receipt'].apply(clean_org_name)
sales = sales.drop(columns=['organization_id_product', 'id_receipt'])
sales = sales.rename(columns={'name_product': 'product_name', 'price_product': 'product_price', 
                              'category_product': 'product_category', 'organization_id_receipt': 'organization_id',
                              'price_receipt': 'receipt_price', 'category_receipt': 'receipt_category',
                              'name_receipt': 'org_name', 'municipality': 'city'})

In [23]:
sales['month'] = sales['date'].dt.month
sales['year'] = sales['date'].dt.year
sales['month_year'] = sales['date'].dt.to_period('M').astype(str)
sales['weekday'] = sales['date'].dt.day_name()
sales['all_item_price'] = sales['product_price'] * sales['quantity']

In [24]:
import sqlite3
conn = sqlite3.connect('sales.db')
sales.to_sql('sales', conn, if_exists='replace', index=False)
conn.close()

In [25]:
sales.head(5)

,id_item,quantity,product_id,receipt_id,product_name,item_type,product_price,product_category,organization_id,date,receipt_price,receipt_category,org_name,country,city,street_name,month,year,month_year,weekday,all_item_price
0,1,1.0,1,1,"NESTEA CITRÓN 1,5l",K,0.99,drinks/non-alcoholic,1,2022-04-02 15:09:57,39.56,Stravovanie/null,BILLA,Slovensko,Bratislava,Bajkalská,4,2022,2022-04,Saturday,0.99
1,2,6.0,2,1,ZÁLOHA PET,K,0.15,drinks/non-alcoholic,1,2022-04-02 15:09:57,39.56,Stravovanie/null,BILLA,Slovensko,Bratislava,Bajkalská,4,2022,2022-04,Saturday,0.90
2,3,1.0,3,1,"MAG.GREP 1,5l",K,0.65,drinks/non-alcoholic,1,2022-04-02 15:09:57,39.56,Stravovanie/null,BILLA,Slovensko,Bratislava,Bajkalská,4,2022,2022-04,Saturday,0.65
3,4,1.0,4,1,"MAG.PER.1,5l",K,0.59,drinks/mineral-water,1,2022-04-02 15:09:57,39.56,Stravovanie/null,BILLA,Slovensko,Bratislava,Bajkalská,4,2022,2022-04,Saturday,0.59
4,5,1.0,5,1,Raciol rastl.1l,K,2.59,basic/oil-acid-grease,1,2022-04-02 15:09:57,39.56,Stravovanie/null,BILLA,Slovensko,Bratislava,Bajkalská,4,2022,2022-04,Saturday,2.59


In [26]:
sales.columns

Index(['id_item', 'quantity', 'product_id', 'receipt_id', 'product_name',
       'item_type', 'product_price', 'product_category', 'organization_id',
       'date', 'receipt_price', 'receipt_category', 'org_name', 'country',
       'city', 'street_name', 'month', 'year', 'month_year', 'weekday',
       'all_item_price'],
      dtype='object')

In [27]:
sales.describe()

,id_item,quantity,product_id,receipt_id,product_price,organization_id,date,receipt_price,month,year,all_item_price
count,7925.000000,7925.000000,7925.000000,7925.000000,7925.000000,7925.000000,7925,7925.000000,7925.000000,7925.000000,7925.000000
mean,4382.074196,1.960738,3081.809842,475.253123,3.225336,37.113691,2024-02-08 15:41:51.134384896,57.218319,6.244921,2023.629274,4.262968
min,1.000000,0.020000,1.000000,1.000000,0.000000,1.000000,2022-01-18 16:12:53,0.000000,1.000000,2022.000000,0.000000
25%,2354.000000,1.000000,1232.000000,226.000000,0.900000,3.000000,2023-11-20 10:54:44,24.820000,3.000000,2023.000000,1.190000
50%,4432.000000,1.000000,2974.000000,491.000000,1.591615,3.000000,2024-03-01 10:27:22,46.800000,6.000000,2024.000000,1.990000
75%,6425.000000,2.000000,4812.000000,701.000000,2.990000,41.000000,2024-06-21 18:34:00,75.480000,9.000000,2024.000000,3.740000
max,8410.000000,1000.000000,6751.000000,998.000000,422.560000,277.000000,2024-11-05 11:23:00,454.520000,12.000000,2024.000000,422.560000
std,2368.090019,11.712725,2002.091412,286.872772,8.234924,61.474346,NaN,46.186792,3.314067,0.569590,11.276968
